In [24]:
import pandas as pd
import pathlib

DATA_PATH: pathlib.Path = pathlib.Path("..\Data\dataset_train.csv")
df: pd.DataFrame = pd.read_csv(DATA_PATH)

## Notes:

- This takes 44s to read must fix
- Try to just save it to parquet then use some other lazy tool for loading and processing
- For preprocessing we need:
  - Classification:
    - convert all points before failures to 1 else 0 (2h) [Keep Last failure for test]
    - Use under sampling (Imbalanced learn) to create full data (Train test)
  - Regression:
    - Create a column for hours from failure
    - Split the data
  - CNN (If time is available):
    - convert all points before failures to 1 else 0 (2h) [Keep Last failure for test]
    - create a 3D array of chunks (n, seq, 22) -> n = len(df) // seq
    - target will be next value

## Results:
- 1.7GB -> 400MB RAM Usage
- 1.6GB -> 130MB Storage

In [25]:
df.head(10)

,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Flowmeter,Motor_current,COMP,...,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,gpsLong,gpsLat,gpsSpeed,gpsQuality
0,2022-01-01 06:00:00,-0.012,9.758,9.760,-0.028,1.576,63.350,19.049625,3.9550,1,...,1,1,0,0,0,0,-8.65934,41.2124,0,1
1,2022-01-01 06:00:01,-0.012,9.760,9.760,-0.028,1.578,63.250,19.049625,4.0275,1,...,1,1,0,0,0,0,-8.65934,41.2124,0,1
2,2022-01-01 06:00:02,-0.010,9.760,9.760,-0.028,1.578,63.325,19.040281,3.9450,1,...,1,1,0,0,0,0,-8.65934,41.2124,0,1
3,2022-01-01 06:00:03,-0.012,9.756,9.756,-0.030,1.576,63.200,19.040281,3.9300,1,...,1,1,0,0,0,0,-8.65934,41.2124,0,1
4,2022-01-01 06:00:04,-0.012,9.756,9.756,-0.030,1.578,63.150,19.049625,3.9950,1,...,1,1,0,0,0,0,-8.65934,41.2124,0,1
5,2022-01-01 06:00:05,-0.012,9.754,9.754,-0.030,1.576,63.075,19.049625,3.9950,1,...,1,1,0,0,0,0,-8.65934,41.2124,0,1
6,2022-01-01 06:00:06,-0.012,9.754,9.754,-0.028,1.578,63.000,19.049625,3.9475,1,...,1,1,0,0,0,0,-8.65935,41.2124,0,1
7,2022-01-01 06:00:07,-0.010,9.752,9.754,-0.028,1.576,63.075,19.040281,4.0225,1,...,1,1,0,0,0,0,-8.65935,41.2124,0,1
8,2022-01-01 06:00:08,-0.012,9.752,9.752,-0.028,1.576,63.075,19.049625,3.9525,1,...,1,1,0,0,0,0,-8.65935,41.2124,0,1
9,2022-01-01 06:00:09,-0.010,9.750,9.750,-0.028,1.576,63.000,19.049625,3.9250,1,...,1,1,0,0,0,0,-8.65935,41.2124,0,1


In [26]:
df.describe()

,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Flowmeter,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,gpsLong,gpsLat,gpsSpeed,gpsQuality
count,1.077359e+07,1.077359e+07,1.077359e+07,1.077359e+07,1.077359e+07,1.077359e+07,1.077359e+07,1.077359e+07,1.077359e+07,1.077359e+07,1.077359e+07,1.077359e+07,1.077359e+07,10773588.0,1.077359e+07,1.077359e+07,1.077359e+07,1.077359e+07,1.077359e+07,1.077359e+07
mean,1.152184e+00,8.974608e+00,7.751421e+00,-2.454095e-02,1.565051e+00,6.730720e+01,2.039515e+01,2.383179e+00,8.698926e-01,1.301137e-01,9.347704e-01,8.698921e-01,6.281380e-03,0.0,2.784588e-07,1.488919e-03,-4.384534e+00,2.091144e+01,4.913657e+00,5.076832e-01
std,3.075296e+00,7.006960e-01,3.051447e+00,1.486569e-01,9.016339e-02,5.383852e+00,3.743607e+00,2.193381e+00,3.364215e-01,3.364285e-01,2.469306e-01,3.364221e-01,7.900585e-02,0.0,5.276919e-04,3.855778e-02,4.317794e+00,2.059254e+01,1.151822e+01,4.999410e-01
min,-3.000000e-02,6.000000e-03,-3.400000e-02,-3.800000e-02,1.350000e+00,1.387500e+01,1.883472e+01,-1.250000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,-9.130040e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,-8.000000e-03,8.484000e+00,8.232000e+00,-3.200000e-02,1.470000e+00,6.367500e+01,1.901225e+01,2.500000e-03,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,-8.658910e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,-8.000000e-03,8.984000e+00,8.746000e+00,-2.800000e-02,1.590000e+00,6.832500e+01,1.904028e+01,3.705000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,-8.542650e+00,4.115190e+01,0.000000e+00,1.000000e+00
75%,-6.000000e-03,9.492000e+00,9.290000e+00,-2.600000e-02,1.638000e+00,7.107500e+01,1.925519e+01,3.837500e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,4.118820e+01,0.000000e+00,1.000000e+00
max,1.087600e+01,1.040800e+01,1.041400e+01,8.326000e+00,2.054000e+00,9.790000e+01,4.307241e+01,9.685000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.0,1.000000e+00,1.000000e+00,0.000000e+00,4.194900e+01,3.230000e+02,1.000000e+00


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10773588 entries, 0 to 10773587
Data columns (total 21 columns):
 #   Column           Dtype  
---  ------           -----  
 0   timestamp        object 
 1   TP2              float64
 2   TP3              float64
 3   H1               float64
 4   DV_pressure      float64
 5   Reservoirs       float64
 6   Oil_temperature  float64
 7   Flowmeter        float64
 8   Motor_current    float64
 9   COMP             int64  
 10  DV_eletric       int64  
 11  Towers           int64  
 12  MPG              int64  
 13  LPS              int64  
 14  Pressure_switch  int64  
 15  Oil_level        int64  
 16  Caudal_impulses  int64  
 17  gpsLong          float64
 18  gpsLat           float64
 19  gpsSpeed         int64  
 20  gpsQuality       int64  
dtypes: float64(10), int64(10), object(1)
memory usage: 1.7+ GB


In [29]:
df_processed: pd.DataFrame = df.drop(columns=["Pressure_switch", "gpsLat", "gpsLong", "gpsQuality"])\
    .astype({
        "timestamp" : "datetime64",
        "TP2" : "float16",
        "TP3" : "float16",
        "H1"  : "float16",
        "DV_pressure" : "float16",
        "Reservoirs" : "float16",
        "Oil_temperature" : "float16",
        "Flowmeter" : "float16",
        "Motor_current" : "float16",
        "COMP" : "bool",
        "DV_eletric" : "bool",
        "Towers" : "bool",
        "MPG" : "bool",
        "LPS" : "bool",
        "Oil_level" : "bool",
        "Caudal_impulses" : "bool",
        "gpsSpeed" : "int16",   
    })

In [30]:
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10773588 entries, 0 to 10773587
Data columns (total 17 columns):
 #   Column           Dtype         
---  ------           -----         
 0   timestamp        datetime64[ns]
 1   TP2              float16       
 2   TP3              float16       
 3   H1               float16       
 4   DV_pressure      float16       
 5   Reservoirs       float16       
 6   Oil_temperature  float16       
 7   Flowmeter        float16       
 8   Motor_current    float16       
 9   COMP             bool          
 10  DV_eletric       bool          
 11  Towers           bool          
 12  MPG              bool          
 13  LPS              bool          
 14  Oil_level        bool          
 15  Caudal_impulses  bool          
 16  gpsSpeed         int16         
dtypes: bool(7), datetime64[ns](1), float16(8), int16(1)
memory usage: 339.1 MB


In [33]:
OUT_PATH: pathlib.Path = pathlib.Path("../Data/")
df_processed.to_feather(OUT_PATH / "Processed.feather")